In [1]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import chi2_contingency, beta 

## 1. Experiment Definition


Developed a new webpage and want to test it's effects on purchase conversion. As such we split our users evenly into 2 groups:

Control: They get the old webpage

Treatment: They get the new webpage

Metric we want to track:

                            purchase conversion= no. of converted users / no. of exposed users

Exposure: A user is bucketed as control or treatment and sees their corresponding page for the first time in the experiment duration

Conversion: An exposed user makes a purchase within 7 days of being first exposed

We have 3 weeks of logged exposure/conversion data.

## 2. Data Collection

 https://www.kaggle.com/saraabdelaal/abtestdata
 

In [4]:
df=pd.read_csv('ab_data.csv')

In [5]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [8]:
df['timestamp']=pd.to_datetime(df['timestamp'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   user_id       294478 non-null  int64         
 1   timestamp     294478 non-null  datetime64[ns]
 2   group         294478 non-null  object        
 3   landing_page  294478 non-null  object        
 4   converted     294478 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 11.2+ MB


In [11]:
start_time =df['timestamp'].min()
end_time = df['timestamp'].max()
data_duration = (end_time - start_time).days

print(f"Number of unique users in experiment: {df['user_id'].nunique()}")
print(f"Data collected for {data_duration} days")
print(f"Landing pages to compare: {df['landing_page'].unique().tolist()}")
print(f"Percentage of users in control: {round(df[df['group']=='control'].shape[0] * 100 / df.shape[0])}%")

Number of unique users in experiment: 290584
Data collected for 21 days
Landing pages to compare: ['old_page', 'new_page']
Percentage of users in control: 50%


## 3. Data Preprocessing

In [28]:
counter = df['user_id'].value_counts()
(counter > 1).value_counts()

False    286690
True       3894
Name: user_id, dtype: int64

In [21]:
#Get timestamp of first exposure
#Remove users with multiple buckets


#first_conversion = df.groupby('user_id')['timestamp'].min().to_frame().reset_index()
#df = df.merge(first_conversion, on=['user_id', 'timestamp'])

3894 (1.32 %) user_ids have been exposed to the old AND new page. It should be okay to remove them

In [33]:
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])
df = df.merge(valid_users, on=['user_id'])

,user_id
733576,1
835407,1
816331,1
871573,1
768544,1
...,...
642985,1
771499,1
923606,1
712675,1


In [43]:
# Add week column to see the data as you would during experiment
df['week'] = df['timestamp'].dt.week
df.head()

C:\Users\taniyas\AppData\Local\Temp/ipykernel_14296/1630598810.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['timestamp'].dt.week


,user_id,timestamp,group,landing_page,converted,week
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,3
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,2
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,2
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,3


In [44]:
df['week'].value_counts()

2    93853
3    93518
1    86058
4    21049
Name: week, dtype: int64

## 4. Experiment: Frequentist Approach

In [46]:
# Vary number to get experiment data at weekly points in time
NUM_WEEKS = 4
experiment_data = df[df['week'] <= NUM_WEEKS]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round(treatment_conversion_perc - control_conversion_perc, 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.892%
Control Conversion Rate: 12.04%
Lift = -0.148%


## Chi-Squared Test

H0 = control and treatment are independent

H1 = control and treatment are not independent

In [47]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted
contingency_table = np.array([[control_converted, control_non_converted], 
                              [treatment_converted, treatment_non_converted]])
contingency_table

array([[ 17723, 129479],
       [ 17514, 129762]], dtype=int64)

In [48]:
chi, p_value, _, _ = chi2_contingency(contingency_table, correction=False)
chi, p_value

(1.5299754273296697, 0.21611613269757673)

Since the p_value > 0.05, we cannot reject null hypothesis. Hence, we cannot conclude if there exists a relationship between the control and treatment groups.

In [49]:
print(f"{round(p_value * 100, 2)}% probability that a more extreme chi square than {round(chi, 3)} would have occured by chance")

21.61% probability that a more extreme chi square than 1.53 would have occured by chance


But this is tough to interpret. We would to say something about the actual maginitude of lift. Something like this:

We are 21.61% confident that our lift = -0.148%

thats why we use Bayesian Approach

## 5. Experiment: Bayesian Approach

We want to input the prior distribution and have the experiment update the parameters to create posterier distributions. Since these prior & posterior distributions will be used to sample Conversion Rate, we model them after beta distribtion.

Let's create the prior beta distribtion from the first weeks of conversion data

In [51]:
prior = df[(df['week'] == 1) & (df['group']=='control')]

In [60]:
prior_means = []
for i in range(10000):
    prior_means.append(prior.sample(1000)['converted'].mean())

prior_means[:10]

[0.124, 0.138, 0.12, 0.129, 0.135, 0.121, 0.11, 0.123, 0.109, 0.123]

In [61]:
# Model Beta Distribtion from sample means
prior_alpha, prior_beta, _, _ = beta.fit(prior_means, floc=0, fscale=1)

In [87]:
# Get Stats
NUM_WEEKS = 4 # Vary number to get experiment data at weekly points in time
experiment_data = df[(df['week'] > 1) & (df['week'] <= NUM_WEEKS)]
control = experiment_data[experiment_data['group']=='control']
treatment = experiment_data[experiment_data['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round((treatment_conversion_perc - control_conversion_perc) / control_conversion_perc , 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.936%
Control Conversion Rate: 12.082%
Lift = -0.012%


In [66]:
control_converted = control['converted'].sum()
treatment_converted = treatment['converted'].sum()
control_non_converted = control['converted'].count() - control_converted
treatment_non_converted = treatment['converted'].count() - treatment_converted

# Update Prior parameters with experiment conversion rates
posterior_control = beta(prior_alpha + control_converted, prior_beta + control_non_converted)
posterior_treatment = beta(prior_alpha + treatment_converted, prior_beta + treatment_non_converted)

# Sample from Posteriors
control_samples = posterior_control.rvs(1000)
treatment_samples = posterior_treatment.rvs(1000)
probability = np.mean(treatment_samples > control_samples)
print(f"Probability that treatment > control: {probability * 100}%")

Probability that treatment > control: 15.4%


In [71]:
(control_mu), (control_var) = posterior_control.stats()
(treatment_mu), (treatment_var) = posterior_treatment.stats()
print(f"Control Posterior: Mean: {control_mu}, Variance: {control_var}") 
print(f"Treatment Posterior: Mean: {treatment_mu}, Variance: {treatment_var}")

Control Posterior: Mean: 0.12080943598142307, Variance: 1.008559431710468e-06
Treatment Posterior: Mean: 0.11936121695553639, Variance: 9.998194758363268e-07


In [80]:
lift_percentage = (treatment_samples - control_samples) / control_samples
lift_percentage.mean()

-0.012130933699964501

In [81]:
print(f"Probability that we are seeing a 1% lift: {np.mean((100 * lift_percentage) > 1) * 100}%")

Probability that we are seeing a 1% lift: 3.5000000000000004%


Advantages of Bayesian over Frequentist:

Results are more interpretable than the ones we got from the frequentist approach

We can interpret results at any point during the experiment. Don't need to wait for an arbitrary "statsig"